<a href="https://colab.research.google.com/github/SarahEverke/SarahEverke-IIC3633-2020/blob/master/practicos/9.FastFM_factorization_machines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctico librería fastFM  - Factorization Machines

Clase: IIC3633 Sistemas Recomendadores, PUC Chile

En este práctico vamos a utilizar la biblioteca de Python [fastFM](https://github.com/ibayer/fastFM) para recomendación utilizando máquinas de factorización. 

En este caso utilizaremos un dataset de cervezas, donde además de incluir interacciones de usuarios con los items agregaremos feature de tipo de cerveza. (style-id)

In [1]:
!curl -L -o "beer_data.base" "https://docs.google.com/uc?export=download&id=1yp9UpqPCESNySlWlDoSEau5aBNKx0nYB"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    604      0 --:--:-- --:--:-- --:--:--   604
100  775k  100  775k    0     0   964k      0 --:--:-- --:--:-- --:--:-- 94.6M


In [2]:
!pip3 install fastFM

     |████████████████████████████████| 491kB 3.4MB/s 


In [3]:
import numpy as np
import pandas as pd
import fastFM
from fastFM.datasets import make_user_item_regression
from sklearn.model_selection import train_test_split
from fastFM import sgd
from fastFM import als
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
from fastFM import mcmc
import functools as fct
import itertools as itools
import random, scipy

# Antes de recomendar hacemos un analisis de los datos 

In [5]:
df = pd.read_csv('beer_data.base',  sep=',',encoding='latin-1')
df.head()

,userID,itemID,styleID,rating
0,4924,11757,1199,4.5
1,4924,5441,1199,4.5
2,4924,19960,1199,5.0
3,2916,55900,1199,2.5
4,2916,57110,14879,4.0


In [6]:
num_of_items = len(df['itemID'].unique().tolist())
num_of_users = len(df['userID'].unique().tolist())
num_of_ratings = len(df['userID'])

print('Num. of items: {}\nNum. of users: {}\nNum. of ratings: {}'.format(num_of_items, num_of_users, num_of_ratings))

Num. of items: 1836
Num. of users: 8320
Num. of ratings: 44379


In [7]:
# rating promedio 
df.describe()['rating']

count    44379.000000
mean         3.865105
std          0.712633
min          0.000000
25%          3.500000
50%          4.000000
75%          4.500000
max          5.000000
Name: rating, dtype: float64

In [8]:
# items que han recibido mas ratings
df.itemID.value_counts().head()

11757    2206
19960    1681
16074    1260
5441     1253
429      1183
Name: itemID, dtype: int64

In [9]:
# usuarios que han dado mas rating 
df.userID.value_counts().head()

13     181
24     129
490    115
100    111
695    106
Name: userID, dtype: int64

In [10]:
# estilos que han recibido más ratings 
df.styleID.value_counts().head()

1199     17400
394       3584
14879     2656
263       2104
3268      1503
Name: styleID, dtype: int64

# Convertir a formato fastFM

## funciones 

In [11]:
def get_single_entries_in_fm_input_format(data, itemlist):
    
    '''Cree el formato de entrada necesario (datos, (fila, columna)) para la matriz csc para
    las entradas individuales en los datos. Cada entrada ocuparía una fila. Esto significa que
    daría como resultado una matriz csc con dimensión (| datos | x | lista de elementos |).
    '''
    
    column = len(itemlist)
    row = len(data)
    shape = (row, column)

    row_inds = np.zeros(len(data), dtype=np.int)
    col_inds = np.zeros(len(data), dtype=np.int)
    datalist = np.zeros(len(data), dtype=np.float)
    
    for i in range(len(data)):
        item = data[i]
        val = 1
        datalist[i] = val
        
        # ubica su posición en la lista de elementos, arroja un error si el elemento no es un
        # artículo posible
        col_ind = np.where(itemlist==item)[0]
        
        # no deben ser elementos duplicados en la lista de elementos
        assert len(col_ind) == 1
        col_ind = col_ind[0]
        row_ind = i

        col_inds[i] = col_ind
        row_inds[i] = row_ind

    return datalist, row_inds, col_inds, shape


def get_multi_entries_in_fm_input_format(data, itemlist, norm_func=None):
    
    '''Cree el formato de entrada necesario (datos, (fila, columna)) para la matriz csc para
    las entradas múltiples en los datos. Cada conjunto de entradas múltiples ocuparía una fila.
    Esto significa que daría como resultado una matriz csc con dimensión
    (| conjuntos de entradas en datos | x | lista de elementos |).
    '''
    
    column = len(itemlist)
    
    # número de conjuntos de entradas en los datos
    row = len(data)
    shape = (row, column)

    # numero de datos 
    num_of_data = fct.reduce(lambda x, y: x + len(y), data, 0)
    row_inds = np.zeros(num_of_data, dtype=np.int)
    col_inds = np.zeros(num_of_data, dtype=np.int)
    datalist = np.zeros(num_of_data, dtype=np.float)
    cnt = 0
    for i in range(len(data)):
        multi_entry = data[i]

        if norm_func != None:
            # función que recibe el tamaño del multi_entry para decidir cómo normalizarlo
            val = norm_func(len(multi_entry))
        else:
            # asignación de valor binario por defecto
            val = 1 if len(multi_entry) > 0 else 0

        # para cada entrada en multi_entry, ubique su posición en la lista de elementos,
        # arroja error si el elemento no es un elemento posible
        # todas las entradas permanecen en la misma fila
        row_ind = i
        for item in multi_entry:
            col_ind = np.where(itemlist==item)[0]
            assert len(col_ind) == 1
            col_ind = col_ind[0]
            
            datalist[cnt] = val
            col_inds[cnt] = col_ind
            row_inds[cnt] = row_ind
            
            # actualiza contador
            cnt += 1

    return datalist, row_inds, col_inds, shape


## conversion de los datos 

In [12]:
beerlist = df.sort_values('itemID')['itemID'].unique()
userlist = df.sort_values('userID')['userID'].unique()
stylelist = df.sort_values('styleID')['styleID'].unique()

# usuarios que dieron ratings 
user_data = df['userID'].values

# items que recibieron ratings
beer_data = df['itemID'].values

# data de estilo de cerveza 
styles_data = df['styleID'].values

# target vector: ratings
rating_data = df['rating'].values


# convertir a formato fastFM utilizando funciones de arriba 
user_datalist, user_row_inds, user_col_inds, user_shape = get_single_entries_in_fm_input_format(data=user_data, 
                                                                                                itemlist=userlist)

beer_datalist, beer_row_inds, beer_col_inds, beer_shape = get_single_entries_in_fm_input_format(data=beer_data,
                                                                                                   itemlist=beerlist)

style_datalist, style_row_inds, style_col_inds, style_shape = get_single_entries_in_fm_input_format(data=styles_data,
                                                                                                   itemlist=stylelist)

# Concatena las dos columnas cambiando los índices de las columnas relacionadas con beer.
# cambiar por el número de columnas en las columnas de usuario
shift_by = len(userlist)
beer_col_inds += shift_by
beer_col_inds += shift_by

# concatena los datos (agregamos item_styles)
datalist = np.append(user_datalist, [beer_datalist, style_datalist])
row_inds = np.append(user_row_inds, [beer_row_inds, style_row_inds])
col_inds = np.append(user_col_inds, [beer_col_inds,style_col_inds])

# asegúrese de que ambos conjuntos de características tengan el mismo número de filas
print('User feature set shape: {}\nItem feature set shape: {}\nStyle feature set shape: {}'.format(user_shape, beer_shape, style_shape))

assert user_shape[0] == beer_shape[0]
shape = (user_shape[0], user_shape[0] + beer_shape[0] + style_shape[0])
print('Dimension of FM input: {}'.format(shape))

X = csc_matrix((datalist, (row_inds, col_inds)), shape=shape)
y = rating_data

User feature set shape: (44379, 8320)
Item feature set shape: (44379, 1836)
Style feature set shape: (44379, 210)
Dimension of FM input: (44379, 133137)


In [13]:
# split train y test
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [14]:
# entrenar modelo optimizando con ALS y hacer la prediccion 
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=10, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)
y_pred = fm.predict(X_test)

In [15]:
error_als = mean_squared_error(y_test, y_pred)
print('Mean squared error under ALS: {}'.format(error_als))

Mean squared error under ALS: 0.6690572660218334


In [16]:
# entrenar modelo optimizando con SGD y hacer la prediccion 
fm_sgd = sgd.FMRegression(n_iter=10000000, init_stdev=0.01, rank=10, random_state=123, 
                              l2_reg_w=0.1, l2_reg_V=0.5, step_size=0.01)
fm_sgd.fit(X_train, y_train)
y_pred_sgd = fm_sgd.predict(X_test)

In [ ]:
error_sgd = mean_squared_error(y_test, y_pred_sgd)
print('Mean squared error under SGD: {}'.format(error_sgd))

Mean squared error under SGD: 0.45126971767960844
